In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [16]:
## Read the pdf from folder
loader=PyPDFDirectoryLoader("./us-census")
documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(documents)
final_documents[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us-census\\acsbr-015.pdf', 'page': 0})

In [17]:
len(final_documents)

316

In [19]:
## Embedding using Huggingface
embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    # model_name="sentence-transformers/all-MiniLM-16-v2",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings': True}
)

c:\WORKSPACE\GitHub\langchain-projects\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\WORKSPACE\GitHub\langchain-projects\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\z002t72b\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In or

In [21]:
import numpy as np
print(np.array(embeddings.embed_query(final_documents[0].page_content)))
print(np.array(embeddings.embed_query(final_documents[0].page_content)).shape)

[-8.46568793e-02 -1.19099254e-02 -3.37892286e-02  2.94559114e-02
  5.19159809e-02  5.73839732e-02 -4.10017818e-02  2.74268091e-02
 -1.05128221e-01 -1.58056226e-02  7.94858783e-02  5.64318374e-02
 -1.31765259e-02 -3.41544151e-02  5.81604894e-03  4.72547859e-02
 -1.30746625e-02  3.12989205e-03 -3.44225764e-02  3.08406260e-02
 -4.09086123e-02  3.52738164e-02 -2.43761521e-02 -4.35831398e-02
  2.41503213e-02  1.31986691e-02 -4.84449975e-03  1.92347635e-02
 -5.43912724e-02 -1.42735049e-01  5.15527185e-03  2.93115675e-02
 -5.60810752e-02 -8.53535719e-03  3.14141139e-02  2.76736543e-02
 -2.06188504e-02  8.24231654e-02  4.15425412e-02  5.79655096e-02
 -3.71587090e-02  6.26158621e-03 -2.41390299e-02 -5.61792636e-03
 -2.51715500e-02  5.04966080e-03 -2.52801087e-02 -2.91942735e-03
 -8.24045949e-03 -5.69604561e-02  2.30823122e-02 -5.54221449e-03
  5.11555746e-02  6.09937422e-02  6.49766102e-02 -5.38514033e-02
  2.19109841e-02 -2.54193954e-02 -4.49223109e-02  4.22459133e-02
  4.75252084e-02  7.23226

In [22]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:50], embeddings)

In [23]:
## Query using Similarity Search
query="Tell me about health insurance coverage"
relevant_documents=vectorstore.similarity_search(query)

print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [24]:
retriever=vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000015A561E1390> search_kwargs={'k': 3}


In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [28]:
from langchain_community.llms import HuggingFaceHub

hf= HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1, "max_length":500}
)

query="What is health insurance?"
hf.invoke(query)

KeyboardInterrupt: 

In [29]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

c:\WORKSPACE\GitHub\langchain-projects\venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-663e3d0b-547e3c4437a945424ec853a2;a5c0e35b-b389-4e5e-977c-8c81dcfb21f1)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must be authenticated to access it.

In [30]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [31]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [32]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [33]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [35]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

KeyboardInterrupt: 